This notebook imports the fundamental objects of the streamm.forcefields module and goes through the functionality of each

In [1]:
from pprint import pprint

In [2]:
import logging
logging.basicConfig(filename='forcefield_example.log',level=logging.DEBUG)

In [3]:
import streamm.forcefields.particletype as particletype

Let's start with a methane molecule from the structure.ipynb example 

We have an sp3 carbon bonded to hydrogens

Let's create the force field parameters object for a 'CT' carbon and a HC hydrogen

In [4]:
CT = particletype.Particletype('CT')
HC = particletype.Particletype('HC')

Set some parameters from J. Am. Chem. Soc., Vol. 121, No. 20, 1999

In general you should pick a force field that has been shown to work well for your system and set up the parameters 

Check that we have our units set right

In [5]:
print CT.unit_conf['energy'],CT.unit_conf['length']

Ha ang


Our potential is in kCal/mol (`kCalmol`) so let's get the unit dictionary and create our own defaul

In [6]:
import copy

In [7]:
oplsaa_unit_conf = copy.deepcopy(CT.unit_conf)

In [8]:
oplsaa_unit_conf['energy'] = 'kCalmol'

In [9]:
pprint(oplsaa_unit_conf)

{u'amount': u'atom',
 u'angle': u'degree',
 u'capacitance': u'F',
 u'charge': u'e',
 u'conductance': u'S',
 u'current': u'A',
 u'density': u'amu_nm^3',
 u'electric_dipole_moment': u'D',
 u'emf': u'V',
 u'energy': 'kCalmol',
 u'force': u'GN',
 u'frequency': u'Hz',
 u'harm_bond_coeff': u'kCalmolsqang',
 u'intensity': u'cd',
 u'length': u'ang',
 u'magnetic_flux': u'Wb',
 u'mass': u'amu',
 u'memory': u'Kb',
 u'power': u'GW',
 u'pressure': u'KPa',
 u'resistance': u'ohm',
 u'temperature': u'K',
 u'time': u'ns',
 u'volume': u'nm^3'}


In [10]:
CT.update_units(oplsaa_unit_conf)

In [11]:
HC.update_units(oplsaa_unit_conf)

In [12]:
HC.epsilon = 0.03 # kcal/mol
HC.sigma = 2.5 # Angstroms 

In [13]:
CT.epsilon = 0.066 # kcal/mol
CT.sigma = 3.5 # Angstroms 

Set the bond stretching parameters 

In [14]:
import streamm.forcefields.bondtype as bondtype

In [15]:
C_H = bondtype.Bondtype('CT','HC',unit_conf=oplsaa_unit_conf)

In [16]:
C_H.setharmonic(1.080,367.0)

In [17]:
print C_H

 bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  


In [18]:
import streamm.forcefields.angletype as angletype

In [19]:
H_C_H = angletype.Angletype('HC','CT','HC')

In [20]:
H_C_H.setharmonic(110.7,37.50)

In [21]:
print H_C_H

 angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  


Let's create a parameter container to keep track of our parameters

In [22]:
import streamm.forcefields.parameters as parameters 

In [23]:
paramC = parameters.Parameters('oplsaa',unit_conf=oplsaa_unit_conf)

Add parameters to the container

In [24]:
paramC.add_particletype(CT)

In [25]:
paramC.add_particletype(HC)

In [26]:
paramC.add_bondtype(C_H)

In [27]:
paramC.add_angletype(H_C_H)

In [28]:
print paramC


    Parameters 
      LJ parameters 2 
      Bond parameters 1 
      Angle parameters 1 
      Dihedral parameters 0 
      Imporper Dihedral parameters 0 



In [29]:
for ptkey,pt in paramC.particletypes.iteritems():
    print ptkey,pt

0  CT epsilon:0.066 sigma:3.5
1  HC epsilon:0.03 sigma:2.5


In [30]:
for btkey,bt in paramC.bondtypes.iteritems():
    print btkey,bt

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  


In [31]:
for atkey,at in paramC.angletypes.iteritems():
    print atkey,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  


In [32]:
print paramC.tag

oplsaa


In [33]:
paramC.unit_conf

{u'amount': u'atom',
 u'angle': u'degree',
 u'capacitance': u'F',
 u'charge': u'e',
 u'conductance': u'S',
 u'current': u'A',
 u'density': u'amu_nm^3',
 u'electric_dipole_moment': u'D',
 u'emf': u'V',
 u'energy': 'kCalmol',
 u'force': u'GN',
 u'frequency': u'Hz',
 u'harm_bond_coeff': u'kCalmolsqang',
 u'intensity': u'cd',
 u'length': u'ang',
 u'magnetic_flux': u'Wb',
 u'mass': u'amu',
 u'memory': u'Kb',
 u'power': u'GW',
 u'pressure': u'KPa',
 u'resistance': u'ohm',
 u'temperature': u'K',
 u'time': u'ns',
 u'volume': u'nm^3'}

In [34]:
print paramC.dump_pickle()

None


Read in methane .xyz file from the structures example

In [35]:
import streamm.structures.buildingblock as bb

In [36]:
mol_i = bb.Buildingblock('methane')

In [37]:
mol_i.read_xyz()

Let's set the ffkey for each particle based on the symbol. 

In [44]:
for pk,p in mol_i.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CA'
    elif( p.symbol == 'H' ):
        p.paramkey == 'HA' 
    print p.ffkey ,mol_i.bonded_nblist.calc_nnab(pk)


CA 4
HA 1
HA 1
HA 1
HA 1


This is a bit redundant, but we can think of a more complex molecule where we could use the number of neighbors to write a more complex routine 

Find neighbor list based on bonded radius 

In [39]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Find bonds and bond angles based on neighbor list 

In [40]:
mol_i.bonded_bonds()
mol_i.bonded_angles()

In [41]:
print mol_i.n_particles

5


Now we can set the particles, bonds and bond angles of the molecule to have parameters

First lets set the particle types 

In [45]:
for pk,p in mol_i.particles.iteritems():
    if( p.paramkey == 'CA' ):
        p.param = CT
        p.param_index = 0
    elif( p.paramkey == 'HA' ):
        p.param = HC
        p.param_index = 1


atom[0] C (C)  CT epsilon:0.066 sigma:3.5 0
atom[1] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[2] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[3] H (H)  HC epsilon:0.03 sigma:2.5 1
atom[4] H (H)  HC epsilon:0.03 sigma:2.5 1


Now we can set the bond types

In [46]:
for bk,b in mol_i.bonds.iteritems():
    b.param = C_H
    b.param_index = 0 

In [47]:
for ak,a in mol_i.angles.iteritems():
    a.param = H_C_H
    b.param_index = 0 

In [ ]:
for pk,p in mol_i.particles.iteritems():
    print p,p.param, p.param_index 

Now our molecule has forcefield paramters for all the interactions 

Sweet as, bro!